In [75]:
# Select coastal points (2+ longitudinal points, average one per lat, and annually
# note that this are daily SST anomalies from OISST  V2 high resolutiton dataset

# output file is used in OISST_coastal_trends notebook script

In [78]:
# Modules
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import datetime as dt
import fsspec
import s3fs
import scipy.stats as stats
# make datasets display nicely
xr.set_options(display_style="html")  

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'


In [66]:
coast = pd.read_csv('coastal_line.csv', index_col=0)
coast = coast.sort_values(by='Lat')
coast = coast[coast['Lat']>=31]
coast = coast[coast['Lat']<=43]
coast = coast.reset_index(drop=True)
lats = np.unique(coast['Lat'])
coast

,Lat,Lon
0,31.125,243.625
1,31.125,243.375
2,31.375,243.625
3,31.375,243.375
4,31.625,243.375
...,...,...
106,42.375,235.625
107,42.625,235.375
108,42.625,235.625
109,42.875,235.375


In [69]:
ds_sst = xr.open_mfdataset('./OISST*.nc')
iy=ds_sst['time.year'].min().data
fy=ds_sst['time.year'].max().data
ds_sst

<xarray.Dataset>
Dimensions:  (lat: 48, lon: 52, time: 14610)
Coordinates:
  * lat      (lat) float32 31.125 31.375 31.625 31.875 ... 42.375 42.625 42.875
  * lon      (lon) float32 232.125 232.375 232.625 ... 244.375 244.625 244.875
  * time     (time) datetime64[ns] 1982-01-01 1982-01-02 ... 2021-12-31
Data variables:
    anom     (time, lat, lon) float32 dask.array<chunksize=(1461, 48, 52), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.5
    title:          NOAA High-resolution Blended Analysis: Daily Values using...
    institution:    NOAA/NCDC
    source:         NOAA/NCDC  ftp://eclipse.ncdc.noaa.gov/pub/OI-daily-v2/
    comment:        Reynolds, et al., 2007: Daily High-Resolution-Blended Ana...
    history:        Tue Jan 25 17:17:57 2022: ncrcat -O -d lon,232.000000,245...
    dataset_title:  NOAA Daily Optimum Interpolation Sea Surface Temperature
    References:     https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2....
    NCO:            netCDF Operators version 4.8.1 (Homepage = http://nco.sf....

In [74]:
assts = np.full((fy-iy+1,len(lats)),np.nan)
ulon = list()
for ii,i in enumerate(lats):
    tmp = coast[coast.Lat==i]
    lons = tmp.Lon.values   
    print(lons)
    sst = ds_sst['anom'].sel(time = slice(str(iy)+'-01-01',str(fy)+'-12-31'),
                                                lat  = i,
                                                lon  = slice(lons.min(),lons.max()),
                                               )
    sst = sst.mean(dim='lon')
    ssta=sst.resample(time='12MS').mean()
    assts[:,ii]=ssta.data
    ulon.append(lons.max())
fechas = ssta.time

[243.625 243.375]
[243.625 243.375]
[243.375 243.125]
[242.875 243.125]
[243.125 242.875]
[242.875 242.625]
[242.875 242.625]
[242.375 242.625]
[242.375 242.625]
[242.375 242.125]
[242.125 241.625 241.125 241.375 241.875]
[241.375 240.875 241.125 241.625]
[240.625 240.375 240.125 239.875 239.625 240.875]
[240.125 239.875 239.625 239.375 240.625 240.375]
[239.125 239.375]
[239.125 239.375]
[239.125 239.375]
[239.125 238.875]
[238.875 238.625]
[238.625 238.375]
[238.375 238.125]
[238.125 237.875]
[238.125 237.875]
[237.625 238.125 237.875]
[237.625 237.375]
[237.625 237.375]
[237.375 237.125]
[237.125 236.875 237.375]
[236.875 236.625]
[236.875 236.625]
[236.625 236.375]
[236.375 236.125]
[235.875 236.125]
[235.875 236.125]
[236.125 235.875]
[235.875 236.125]
[235.875 235.625]
[235.625 235.375]
[235.625 235.375]
[235.625 235.875]
[235.625 235.875]
[235.625 235.875]
[235.875 235.625]
[235.375 235.625]
[235.625 235.375]
[235.375 235.625]
[235.375 235.625]
[235.375 235.125]


In [76]:
coastSST = xr.Dataset(data_vars=dict(SST=(['time','point'],assts),
                                    Lon=(['point'],ulon),
                                    Lat=(['point'],lats)),
                      coords=dict(point=([*range(len(lats))]),
                                  time=(fechas)))
coastSST

<xarray.Dataset>
Dimensions:  (point: 48, time: 40)
Coordinates:
  * point    (point) int64 0 1 2 3 4 5 6 7 8 9 ... 38 39 40 41 42 43 44 45 46 47
    lat      float32 42.875
  * time     (time) datetime64[ns] 1982-01-01 1983-01-01 ... 2021-01-01
Data variables:
    SST      (time, point) float64 -0.6573 -0.7083 -0.4031 ... -1.622 -1.087
    Lon      (point) float64 243.6 243.6 243.4 243.1 ... 235.6 235.6 235.6 235.4
    Lat      (point) float64 31.12 31.38 31.62 31.88 ... 42.12 42.38 42.62 42.88

In [77]:
coastSST.to_netcdf('coastal_annual_OISST_1982-2021.nc')